In [2]:
# !pip install python-binance
from datetime import datetime
import pandas as pd

In [2]:
api_key = "nz3BiMJaQubfXwLFAT5z9hCb8RaBj6ec0ddNq88dhI63ADawKrnebrRfE8xtAW7j"
api_secret = "rbTzXhZSllDqI2kfHgYL4IrfYcWHm3tFLHnWMDItMuIQYy3pVLzjHBaouaFoW3wX"

from binance.client import Client
from datetime import datetime
import pandas as pd

COLUMNS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 
                   'tb_base_av', 'tb_quote_av', 'ignore']

client = Client(api_key=api_key, api_secret=api_secret)

def get_data(start_date, end_date, symbol):
    data = client.futures_historical_klines(
            symbol=symbol,
            interval='1m',
            start_str=start_date,
            end_str=end_date
        )
    
    # 전처리
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['open_time'] = df['open_time'].apply(lambda x: datetime.fromtimestamp(x // 1000))
    df = df.drop(columns=['close_time', 'ignore'])
    df['symbol'] = symbol
    df.loc[:, 'open':'tb_quote_av'] = df.loc[:, 'open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    return df

start_date = '2023-10-14'
end_date = '2024-01-15'
symbol = ['BTCUSDT']
for i in symbol:
    df = get_data(start_date=start_date, end_date=end_date, symbol=i)
    df.to_csv(f'3m_{i}.csv', index=False)

---
- 3개의 코인데이터 csv 시계열 데이터로 합치는 코드

In [4]:
import pandas as pd

# 각 비트코인 종목 데이터 파일 경로
six_m_BTCUSDT_path = 'C:\\education\\ml_dev\\coin\jh_-\\binance\\6m_BTCUSDT.csv'
six_m_ETHUSDT_path = 'C:\\education\\ml_dev\\coin\jh_-\\binance\\6m_ETHUSDT.csv'
six_m_SOLUSDT_path = 'C:\\education\\ml_dev\\coin\jh_-\\binance\\6m_SOLUSDT.csv'

# CSV 파일을 데이터프레임으로 읽기
BTCUSDT_df = pd.read_csv(six_m_BTCUSDT_path)
ETHUSDT_df = pd.read_csv(six_m_ETHUSDT_path)
SOLUSDT_df = pd.read_csv(six_m_SOLUSDT_path)

# 시계열 데이터를 하나의 데이터프레임으로 합치기
merged_df = pd.concat([BTCUSDT_df, ETHUSDT_df, SOLUSDT_df])

# 합친 데이터프레임을 CSV 파일로 저장하기
merged_df.to_csv('merged_bitcoin_data.csv', index=False)

In [9]:
test1_df = pd.read_csv('bitcoin_6m.csv')

In [10]:
test1_df.head()

,open_time,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,symbol
0,2023-07-14 09:00:00,31441.6,31451.1,31437.1,31440.0,409.895,1.288859e+07,3691,171.229,5.384128e+06,BTCUSDT
1,2023-07-14 09:01:00,31440.1,31448.0,31436.5,31445.6,205.267,6.454019e+06,2242,139.739,4.393711e+06,BTCUSDT
2,2023-07-14 09:02:00,31445.5,31459.5,31428.2,31459.4,324.958,1.021708e+07,3310,146.179,4.596811e+06,BTCUSDT
3,2023-07-14 09:03:00,31459.5,31459.5,31437.9,31439.9,181.418,5.705181e+06,1939,41.867,1.316560e+06,BTCUSDT
4,2023-07-14 09:04:00,31439.9,31440.0,31416.0,31417.4,443.662,1.394171e+07,3529,137.522,4.321646e+06,BTCUSDT


In [11]:
test1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799203 entries, 0 to 799202
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   open_time    799203 non-null  object 
 1   open         799203 non-null  float64
 2   high         799203 non-null  float64
 3   low          799203 non-null  float64
 4   close        799203 non-null  float64
 5   volume       799203 non-null  float64
 6   quote_av     799203 non-null  float64
 7   trades       799203 non-null  int64  
 8   tb_base_av   799203 non-null  float64
 9   tb_quote_av  799203 non-null  float64
 10  symbol       799203 non-null  object 
dtypes: float64(8), int64(1), object(2)
memory usage: 67.1+ MB


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 데이터 불러오기
data = pd.read_csv('bitcoin_data.csv')

# 타겟값 설정 (5분 후 종가)
data['target'] = data['close'].shift(-5)

# NaN 값이 있는 행 삭제
data.dropna(inplace=True)

# 입력 변수와 타겟 변수 분리
X = data.drop(columns=['open_time', 'close', 'symbol', 'target'])
y = data['target']

# 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [ ]:
# LSTM 모델 생성
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

# 테스트 데이터로 예측
predictions = model.predict(X_test)

# 예측 결과 역정규화
predictions = scaler.inverse_transform(predictions)

# 예측 결과 출력
print(predictions)

# 테스트 데이터의 실제 값 출력
print(y_test)